In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

Các đặc trưng:
1. URL chứa địa chỉ IP
2. Độ dài URL(>=54 nghi ngờ, >=75 chắc chắn)
3. Độ dài URL rất ngắn
4. Sự tồn tại của ký tự '@' trong URL
5. Vị trí cuối cùng của '// trong URL dưới 7(XEM XÉT)
6. Số lượng ký tự '−' trong Tên miền
7. Số dấu chấm trong url (>= 3)
8. Sử dụng https bởi các nhà cung cấp không đáng tin cậy và Tuổi chứng chỉ nhỏ hơn 1 năm(XEM XÉT)
9. Ngày hết hạn của Tên miền nhỏ hơn hoặc bằng 1
10. Favicon lấy từ Nguồn bên ngoài
11. Số cổng có trạng thái ưu tiên
12. Phần tên miền có mã thông báo HTTP
13. Phần trăm URL yêu cầu (>=22% nghi ngờ, lớn hơn 61% chắc chắn)
14. Phần trăm URL yêu cầu (>=31% nghi ngờ, lớn hơn 67% chắc chắn)
15. Phần trăm các liên kết trong "Meta", Liên kết" và "Tập lệnh" (>=17% nghi ngờ, lớn hơn 81% chắc chắn)
16. “Is Empty”or “about: blank” trong SFH (thuộc tính action URL)
17. Sử dụng dịch vụ "mail()"
18. URL không có Tên máy chủ
19. Chuyển hướng trang <=1
20. Thay đổi trong thanh trạng thái bằng cách di chuột qua
21. Vô hiệu hóa nhấp chuột phải
22. Cửa sổ bật lên của trình duyệt với các hộp văn bản
23. Trang web có sử dụng iframe
24. Tuổi miền <=6 tháng(XEM XÉT)
25. Tên miền không có bản ghi DNS
26. Xếp hạng trang web lớn hơn 100.000
27. Xếp hạng trang nhỏ hơn 0,2
28. Trang web không có chỉ mục của google
29. Số lượng liên kết trỏ đến trang web bằng không
30. Máy chủ lưu trữ có địa chỉ IP lừa đảo hàng đầu

1. #url chứa '\.'

In [54]:
def check_url(url):
    return 1 if '\.' in url else 0

#ví dụ sử dụng
# url1 = "https://example.com"
# url2 = "https://example\.com"

# print(check_url(url1))  # Kết quả: 0
# print(check_url(url2))  # Kết quả: 1

2. #Độ dài phản hồi của một trang web
pip install requests-html

In [55]:
import requests

def response_length(url):
    try:
        response = requests.get(url)
        # Kiểm tra nếu phản hồi thành công
        if response.status_code == 200:
            return len(response.content)  # Độ dài của nội dung phản hồi
        else:
            print("Không thể truy cập trang web. Mã trạng thái:", response.status_code)
            return None
    except requests.exceptions.RequestException as e:
        print("Lỗi kết nối:", e)
        return None

# url = "https://jp.mercaricodoaongou.enterenglishmarket.com"
# length = response_length(url)
# print("Độ dài phản hồi:", length)

3. #kiểm tra subdomain
pip install tldextract

In [56]:
pip install tldextract

In [57]:
import tldextract

def count_subdomains(url):
    # Tách các phần của URL
    extracted = tldextract.extract(url)

    # Lấy danh sách các subdomain
    subdomains = extracted.subdomain.split('.')

    # Đếm số lượng subdomain
    return len(subdomains) if subdomains[0] else 0  # Nếu subdomain đầu tiên không có, trả về 0

# Ví dụ sử dụng
# url1 = "https://hoc.sub2.example.com/path"
# url2 = "https://example.com/path"

# print("Số lượng subdomain của url1:", count_subdomains(url1))  # Kết quả: 2
# print("Số lượng subdomain của url2:", count_subdomains(url2))  # Kết quả: 0

4. tham số delta

In [58]:
from urllib.parse import urlparse, parse_qs

def check_delta_in_url(url):
    # Phân tích URL để lấy các tham số truy vấn
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)

    # Kiểm tra xem có tham số 'delta' trong các tham số truy vấn hay không
    return 1 if 'delta' in query_params else 0

# url1 = "https://fithou.edu.vn/resource?delta=true&id=123"
# url2 = "https://fithou.edu.vn/resource?id=123"

# print(check_delta_in_url(url1))  # Kết quả: 1
# print(check_delta_in_url(url2))  # Kết quả: 0

5. #Số lượng kí tự hex

In [59]:
import re

def hex_char_count(url):
    # Tìm tất cả các ký tự hex (0-9 và a-f)
    hex_chars = re.findall(r'[0-9a-fA-F]', url)
    # Trả về độ dài của danh sách hex_chars, là số lượng ký tự hex trong URL
    return len(hex_chars)

# url = "https://example.com/abc123"
# print(hex_char_count(url))  # Kết quả sẽ là 6

6. #số lượng hex/ độ dài url

In [60]:
import re

def hex_ratio_hostname(url):
    # Loại bỏ giao thức (http, https)
    hostname = re.sub(r'^https?://', '', url).split('/')[0]
    # Đếm số lượng ký tự hex trong hostname
    hex_chars = re.findall(r'[0-9a-fA-F]', hostname)
    hex_count = len(hex_chars)
    # Tính tỷ lệ ký tự hex so với tổng độ dài hostname
    ratio = hex_count / len(hostname) if len(hostname) > 0 else 0
    return ratio

# url = "https://example123abc.com"
# print(hex_ratio_hostname(url))  # Kết quả sẽ là tỷ lệ hex/độ dài của hostname

7. #dùng http

In [61]:
def is_http(url):
    # Kiểm tra nếu URL bắt đầu với "http://"
    return 1 if url.startswith("http://") else 0

# url1 = "http://example.com"
# url2 = "https://example.com"
# print(is_http(url1))  # Kết quả: 1
# print(is_http(url2))  # Kết quả: 0

8. #url chứa địa chỉ ip

In [62]:
import re
from urllib.parse import urlparse
def is_ip_address(url):
    # Lấy phần tên miền (hostname) từ URL
    parsed_url = urlparse(url)
    hostname = parsed_url.hostname

    # Mẫu regex kiểm tra địa chỉ IPv4
    ipv4_pattern = re.compile(r"^(?:[0-9]{1,3}\.){3}[0-9]{1,3}$")
    # Mẫu regex kiểm tra địa chỉ IPv6
    ipv6_pattern = re.compile(r"^\[?[0-9a-fA-F:]+\]?$")

    # Kiểm tra nếu hostname là địa chỉ IPv4 hoặc IPv6
    if hostname and (ipv4_pattern.match(hostname) or ipv6_pattern.match(hostname)):
        return 1
    return 0

# Ví dụ sử dụng
# url = "http://192.168.1.1"
# print(is_ip_address(url))  # Kết quả sẽ là 1 nếu URL chứa địa chỉ IP


9. # Độ dài URL

In [63]:
def url_length(url):
    return len(url)

# Ví dụ sử dụng
# url = "https://example.com"
# print("Độ dài của URL là:", url_length(url))

10. #Độ dài hostname

In [64]:
import urllib.parse

def calculate_hostname_length(url: str) -> int:
    # Trích xuất hostname từ URL
    parsed_url = urllib.parse.urlparse(url)
    hostname = parsed_url.hostname

    if hostname is not None:
        # Tính độ dài của hostname
        return len(hostname)
    else:
        raise ValueError("URL không hợp lệ hoặc không có hostname.")

# # Ví dụ sử dụng
# url = "https://www.example.com/path?query=123"
# hostname_length = calculate_hostname_length(url)
# print(f"Độ dài của hostname '{url}': {hostname_length}")

11. #Sự tồn tại của ký tự '@' trong URL

In [65]:
def contains_at_symbol(url):
    return 1 if "@" in url else 0

# Ví dụ sử dụng
url = "https://example.com"
print(contains_at_symbol(url))  # Kết quả: 0

url_with_at = "https://user@example.com"
print(contains_at_symbol(url_with_at))  # Kết quả: 1

0
1


12. #Số lượng chữ số/ hostname

In [66]:
from urllib.parse import urlparse

def digit_ratio_in_hostname(url):
    # Trích xuất hostname từ URL
    hostname = urlparse(url).hostname
    
    # Kiểm tra nếu hostname là None
    if hostname is None:
        return "URL không hợp lệ hoặc không có hostname."
    
    # Đếm số chữ số trong hostname
    digit_count = sum(char.isdigit() for char in hostname)
    
    # Tính độ dài của hostname
    hostname_length = len(hostname)
    
    # Tính tỷ lệ số chữ số trên độ dài hostname
    if hostname_length == 0:
        return "Hostname không hợp lệ."
    
    digit_ratio = digit_count / hostname_length
    return digit_ratio

# Ví dụ
# url = "https://123example456.com/path?param=value"
# digit_ratio = digit_ratio_in_hostname(url)
# print(digit_ratio)

13. #độ đo entropy

In [67]:
import math
import urllib.parse
from collections import Counter

def calculate_entropy(s: str) -> float:
    # Đếm tần suất của từng ký tự
    char_count = Counter(s)
    total_chars = len(s)

    # Tính entropy
    entropy = -sum((count / total_chars) * math.log2(count / total_chars) for count in char_count.values())
    return entropy

def check_hostname_entropy(url: str) -> float:
    # Trích xuất hostname từ URL
    parsed_url = urllib.parse.urlparse(url)
    hostname = parsed_url.hostname

    if hostname is not None:
        # Tính toán entropy của hostname
        entropy = calculate_entropy(hostname)
        return entropy
    else:
        raise ValueError("Invalid URL")

# Ví dụ sử dụng
# url = "https://www.google.com"
# entropy = check_hostname_entropy(url)
# print(f"Entropy của hostname '{url}': {entropy:.4f}")

14. #tỷ lệ nguyên âm
15.  #tỷ lệ phụ âm


In [68]:
import urllib.parse

def calculate_vowel_ratio(url: str) -> float:
    # Trích xuất hostname từ URL
    parsed_url = urllib.parse.urlparse(url)
    hostname = parsed_url.hostname

    if hostname is not None:
        # Đếm số nguyên âm
        vowels = "aeiouAEIOU"
        vowel_count = sum(1 for char in hostname if char in vowels)
        total_chars = len(hostname)

        # Tính tỷ lệ nguyên âm
        if total_chars > 0:
            ratio = vowel_count / total_chars
            return ratio
        else:
            raise ValueError("Hostname không có ký tự nào.")
    else:
        raise ValueError("URL không hợp lệ.")

# Ví dụ sử dụng
url = "https://www.google.com.tr"
vowel_ratio = calculate_vowel_ratio(url)
print(f"Tỷ lệ nguyên âm trong hostname '{url}': {vowel_ratio:.4f}")

Tỷ lệ nguyên âm trong hostname 'https://www.google.com.tr': 0.2353


In [69]:
import urllib.parse

def calculate_consonant_ratio(url: str) -> float:
    # Trích xuất hostname từ URL
    parsed_url = urllib.parse.urlparse(url)
    hostname = parsed_url.hostname

    if hostname is not None:
        # Đếm số phụ âm
        vowels = "aeiouAEIOU"
        consonant_count = sum(1 for char in hostname if char.isalpha() and char not in vowels)
        total_chars = len(hostname)

        # Tính tỷ lệ phụ âm
        if total_chars > 0:
            ratio = consonant_count / total_chars
            return ratio
        else:
            raise ValueError("Hostname không có ký tự nào.")
    else:
        raise ValueError("URL không hợp lệ.")

# Ví dụ sử dụng
url = "https://www.google.com.tr"
consonant_ratio = calculate_consonant_ratio(url)
print(f"Tỷ lệ phụ âm trong hostname '{url}': {consonant_ratio:.4f}")

Tỷ lệ phụ âm trong hostname 'https://www.google.com.tr': 0.5882


16. #google index

In [70]:
import requests
from bs4 import BeautifulSoup

def check_google_index(url: str) -> bool:
    # Thực hiện tìm kiếm trên Google
    query = f"site:{url}"
    google_search_url = f"https://www.google.com/search?q={query}"

    # Gửi yêu cầu tới Google
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(google_search_url, headers=headers)

    # Phân tích nội dung HTML của trang kết quả
    soup = BeautifulSoup(response.text, 'html.parser')

    # Kiểm tra xem có kết quả nào không
    if "did not match any documents" in soup.text:
        return 1  # Không có chỉ mục
    else:
        return 0   # Đã được lập chỉ mục

# Ví dụ sử dụng
# url = "https://www.sarkariresult.com"  # Thay thế bằng URL bạn muốn kiểm tra
# is_indexed = check_google_index(url)

# if is_indexed:
#     print(f"URL '{url}' chưa được lập chỉ mục trên Google.")
# else:
#     print(f"URL '{url}' đã được lập chỉ mục trên Google.")

17. Số lượng dấu '='

In [71]:
def count_equal_signs(url):
    return url.count('=')

# # Ví dụ sử dụng hàm
# url = "http://example.com/page=param=value"
# equal_sign_count = count_equal_signs(url)
# print(equal_sign_count)

18. #chữ số/ hostname

In [72]:
from urllib.parse import urlparse

def count_digits_in_hostname(url):
    # Trích xuất hostname từ URL
    hostname = urlparse(url).hostname
    
    # Kiểm tra nếu hostname là None
    if hostname is None:
        return "URL không hợp lệ hoặc không có hostname."
    
    # Đếm số chữ số trong hostname
    digit_count = sum(char.isdigit() for char in hostname)
    
    return digit_count

# Ví dụ
# url = "https://123example456.com/path?param=value"
# digit_count = count_digits_in_hostname(url)
# print(digit_count)

19. Số lượng dấu ‘-’ / độ dài hostname

In [73]:
from urllib.parse import urlparse

def hyphen_ratio_in_hostname(url):
    # Trích xuất hostname từ URL
    hostname = urlparse(url).hostname
    
    # Kiểm tra nếu hostname là None
    if hostname is None:
        return "URL không hợp lệ hoặc không có hostname."
    
    # Đếm số dấu '-' trong hostname
    hyphen_count = hostname.count('-')
    
    # Tính độ dài của hostname
    hostname_length = len(hostname)
    
    # Tính tỷ lệ số dấu '-' trên độ dài hostname
    if hostname_length == 0:
        return "Hostname không hợp lệ."
    
    hyphen_ratio = hyphen_count / hostname_length
    return hyphen_ratio

# Ví dụ
# url = "https://e-xample-domain.com/path?param=value"
# hyphen_ratio = hyphen_ratio_in_hostname(url)
# print(hyphen_ratio)

#Tồn tại của ký tự '−' trong Tên (xem xét)

In [74]:
def contains_hyphen(url):
    return 1 if "-" in url else 0

# Ví dụ sử dụng
# url1 = "https://example.com"
# print(contains_hyphen(url1))  # Kết quả: 0

# url2 = "https://my-example.com"
# print(contains_hyphen(url2))  # Kết quả: 1


20. # Số dấu chấm trong url

In [75]:
def count_dots(url):
    return url.count('.')

# Ví dụ sử dụng
# url1 = "https://example.com"
# print("Số lượng dấu chấm:", count_dots(url1))  # Kết quả: 1

# url2 = "http://sub.example.co.uk"
# print("Số lượng dấu chấm:", count_dots(url2))  # Kết quả: 3


#Favicon lấy từ Nguồn bên ngoài **xem xét**
pip install requests beautifulsoup4


In [76]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

def is_external_favicon(url):
    try:
        # Gửi yêu cầu đến URL và lấy nội dung
        response = requests.get(url)
        response.raise_for_status()  # Kiểm tra yêu cầu có thành công không

        # Phân tích HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Tìm thẻ <link> với rel="icon"
        favicon_link = soup.find("link", rel=lambda x: x and 'icon' in x)

        if favicon_link and 'href' in favicon_link.attrs:
            favicon_url = favicon_link['href']
            # Kiểm tra xem favicon có phải là nguồn bên ngoài không
            favicon_domain = urlparse(favicon_url).netloc
            base_domain = urlparse(url).netloc

            if favicon_domain and favicon_domain != base_domain:
                return 1  # Favicon từ nguồn bên ngoài
    except Exception as e:
        print(f"Lỗi khi kiểm tra favicon: {e}")

    return 0  # Không phải nguồn bên ngoài hoặc không tìm thấy favicon

# Ví dụ sử dụng
# url = "https://fithou.edu.vn"
# result = is_external_favicon(url)
# print(f"Kết quả kiểm tra favicon cho {url}: {result}")


21. #Số cổng có trạng thái ưu tiên

In [77]:
from urllib.parse import urlparse

def is_priority_port(url):
    parsed_url = urlparse(url)
    # Kiểm tra cổng và giao thức
    if (parsed_url.port in [80, 443] or parsed_url.port is None) and parsed_url.scheme in ['http', 'https']:
        return 1
    return 0

# Ví dụ
# print(is_priority_port("https://fithou.edu.vn"))     # Kết quả: 1
# print(is_priority_port("https://fithou.edu.vn:8080")) # Kết quả: 0

#Phần tên miền có mã thông báo HTTP

In [78]:
def has_http_token(url):
    # Kiểm tra xem URL có bắt đầu bằng "http://" hoặc "https://" không
    if url.startswith("http://") or url.startswith("https://"):
        return 1  # Có mã thông báo HTTP
    return 0  # Không có mã thông báo HTTP

# Ví dụ sử dụng
# url1 = "http://example.com/resource"
# result1 = has_http_token(url1)
# print(f"Kết quả kiểm tra cho {url1}: {result1}")  # Kết quả: 1

# url2 = "ftp://example.com/resource"
# result2 = has_http_token(url2)
# print(f"Kết quả kiểm tra cho {url2}: {result2}")  # Kết quả: 0

22. # Phần trăm URL yêu cầu

In [79]:
def percentage_of_characters(url):
    if not url:  # Kiểm tra nếu URL rỗng
        return 0

    total_length = len(url)
    letter_count = sum(c.isalpha() for c in url)  # Đếm số ký tự là chữ cái

    # Tính phần trăm
    percentage = (letter_count / total_length) * 100
    return round(percentage, 2)  # Làm tròn tới 2 chữ số thập phân

# Ví dụ sử dụng
# url1 = "http://fithou.edu.vn/resource"
# result1 = percentage_of_characters(url1)
# print(f"Phần trăm chữ cái trong URL '{url1}': {result1}%")

# url2 = "ftp://123.fithou.edu.vn/resource"
# result2 = percentage_of_characters(url2)
# print(f"Phần trăm chữ cái trong URL '{url2}': {result2}%")


#Phần trăm các liên kết trong "Meta", Liên kết" và "Tập lệnh"

In [80]:
import requests
from bs4 import BeautifulSoup

def percentage_of_links(url):
    try:
        # Gửi yêu cầu đến URL và lấy nội dung
        response = requests.get(url)
        response.raise_for_status()  # Kiểm tra yêu cầu có thành công không

        # Phân tích HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Tìm tất cả các thẻ meta, link, và script
        meta_tags = soup.find_all('meta')
        link_tags = soup.find_all('link')
        script_tags = soup.find_all('script')

        # Tổng số thẻ
        total_tags = len(meta_tags) + len(link_tags) + len(script_tags)

        # Đếm số lượng liên kết trong các thẻ
        total_links = len([tag for tag in link_tags if 'href' in tag.attrs])  # Thẻ link với href
        total_links += len([tag for tag in script_tags if 'src' in tag.attrs])  # Thẻ script với src

        # Tính phần trăm
        if total_tags > 0:
            percentage = (total_links / total_tags) * 100
            return round(percentage, 2)  # Làm tròn tới 2 chữ số thập phân
        else:
            return 0  # Nếu không có thẻ nào

    except Exception as e:
        print(f"Lỗi khi kiểm tra URL: {e}")
        return 0  # Trả về 0 nếu có lỗi

# Ví dụ sử dụng
# url = "https://fithou.edu.vn"
# result = percentage_of_links(url)
# print(f"Phần trăm liên kết trong '{url}': {result}%")


#“Is Empty”or “about: blank” trong SFH
(Server Form Handler) là khi người dùng submit 1 form trong thẻ <form> sẽ gửi đến 1 link trong action thì link đó chính là SFH
vì vậy nếu action='' thì có khả năng là phishing

In [81]:
import requests
from bs4 import BeautifulSoup

def check_sfh_empty(url):
    try:
        # Gửi yêu cầu đến URL và lấy nội dung
        response = requests.get(url)
        response.raise_for_status()  # Kiểm tra yêu cầu có thành công không

        # Phân tích HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Tìm tất cả các thẻ <form>
        form_tags = soup.find_all('form')

        # Kiểm tra các thuộc tính action của các thẻ <form>
        for form in form_tags:
            action = form.get('action', '').strip()  # Lấy thuộc tính action
            if action.lower() in ["", "about: blank"]:  # Kiểm tra xem có "Is Empty" hoặc "about: blank" không
                return 1  # Có "Is Empty" hoặc "about: blank"

        return 0  # Không có "Is Empty" hoặc "about: blank"

    except Exception as e:
        print(f"Lỗi khi kiểm tra URL: {e}")
        return 0  # Trả về 0 nếu có lỗi

# Ví dụ sử dụng
# url = "https://fithou.edu.vn"
# result = check_sfh_empty(url)
# print(f"Kết quả kiểm tra SFH cho '{url}': {result}")


#Sử dụng dịch vụ "mail()"
pip install requests

In [82]:
import requests

def check_mail_function(url):
    try:
        # Gửi yêu cầu đến URL và lấy nội dung
        response = requests.get(url)
        response.raise_for_status()  # Kiểm tra yêu cầu có thành công không

        # Kiểm tra xem nội dung có chứa 'mail(' không
        if 'mail(' in response.text:
            return 1  # Có sử dụng dịch vụ mail()
        else:
            return 0  # Không sử dụng dịch vụ mail()

    except Exception as e:
        print(f"Lỗi khi kiểm tra URL: {e}")
        return 0  # Trả về 0 nếu có lỗi

# Ví dụ sử dụng
# url = "https://fithou.edu.vn"
# result = check_mail_function(url)
# print(f"Kết quả kiểm tra sử dụng dịch vụ mail() cho '{url}': {result}")


23. #URL không có Tên máy chủ

In [83]:
from urllib.parse import urlparse

def check_has_hostname(url):
    try:
        # Phân tích URL
        parsed_url = urlparse(url)

        # Kiểm tra xem tên máy chủ (netloc) có tồn tại không
        if parsed_url.hostname:  # Nếu có hostname
            return 1  # Có tên máy chủ
        else:
            return 0  # Không có tên máy chủ
    except Exception as e:
        print(f"Lỗi khi kiểm tra URL: {e}")
        return 0  # Trả về 0 nếu có lỗi

# Ví dụ sử dụng
# url1 = "https://example.com/resource"
# result1 = check_has_hostname(url1)
# print(f"Kết quả kiểm tra tên máy chủ cho '{url1}': {result1}")  # Kết quả: 1

# url2 = "ftp://"
# result2 = check_has_hostname(url2)
# print(f"Kết quả kiểm tra tên máy chủ cho '{url2}': {result2}")  # Kết quả: 0

# url3 = "invalid-url"
# result3 = check_has_hostname(url3)
# print(f"Kết quả kiểm tra tên máy chủ cho '{url3}': {result3}")  # Kết quả: 0

24. Số lượng dấu '/' trên 1 url

In [84]:
from urllib.parse import urlparse

def count_slashes(url):
    parsed_url = urlparse(url)
    path = parsed_url.path  # Lấy phần đường dẫn trong URL
    slash_count = path.count('/')  # Đếm số lượng dấu '/' trong đường dẫn
    return slash_count

# # Ví dụ
# url = "https://www.example.com/path/to/resource/"
# print(count_slashes(url))

#Chuyển hướng trang >1

In [85]:
import requests

def check_redirects(url):
    try:
        # Gửi yêu cầu đến URL và theo dõi chuyển hướng
        response = requests.get(url)

        # Kiểm tra số lượng chuyển hướng
        if len(response.history) > 1:  # Nếu số lượng chuyển hướng <= 1
            return 1  # Trả về 1
        else:
            return 0  # Trả về 0

    except Exception as e:
        print(f"Lỗi khi kiểm tra URL: {e}")
        return 0  # Trả về 0 nếu có lỗi

# Ví dụ sử dụng
# url1 = "https://www.msaaezusshubsnsk.top"  # URL có thể có chuyển hướng
# result1 = check_redirects(url1)
# print(f"Kết quả kiểm tra chuyển hướng cho '{url1}': {result1}")

# url2 = "http://httpbin.org/redirect/1"  # Chuyển hướng 1 lần
# result2 = check_redirects(url2)
# print(f"Kết quả kiểm tra chuyển hướng cho '{url2}': {result2}")

# url3 = "http://httpbin.org/redirect/2"  # Chuyển hướng 2 lần
# result3 = check_redirects(url3)
# print(f"Kết quả kiểm tra chuyển hướng cho '{url3}': {result3}")


Thay đổi trong thanh trạng thái bằng cách di chuột qua

In [86]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [87]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

def check_status_bar_change(url):
    # Thiết lập trình duyệt (sử dụng Chrome)
    driver = webdriver.Chrome()  # Đảm bảo rằng ChromeDriver đã được cài đặt
    driver.get(url)

    # Tìm phần tử cần kiểm tra
    try:
        element = driver.find_element(By.TAG_NAME, 'a')  # Thay đổi thành selector phù hợp với phần tử bạn muốn kiểm tra

        # Lưu trạng thái ban đầu
        original_status = driver.execute_script("return window.status;")

        # Thực hiện di chuột qua phần tử
        ActionChains(driver).move_to_element(element).perform()
        time.sleep(1)  # Đợi một chút để xem có thay đổi không

        # Kiểm tra trạng thái sau khi di chuột qua
        new_status = driver.execute_script("return window.status;")

        if original_status != new_status:  # Nếu trạng thái đã thay đổi
            return 1  # Trả về 1
        else:
            return 0  # Trả về 0
    except Exception as e:
        print(f"Lỗi khi kiểm tra URL: {e}")
        return 0  # Trả về 0 nếu có lỗi
    finally:
        driver.quit()  # Đóng trình duyệt

# Ví dụ sử dụng
# url = "https://www.google.co.in"  # Thay đổi thành URL mà bạn muốn kiểm tra
# result = check_status_bar_change(url)
# print(f"Kết quả kiểm tra thay đổi thanh trạng thái cho '{url}': {result}")


#Vô hiệu hóa nhấp chuột phải
pip install selenium

In [88]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def check_right_click_disabled(url):
    # Thiết lập trình duyệt (sử dụng Chrome)
    driver = webdriver.Chrome()  # Đảm bảo rằng ChromeDriver đã được cài đặt
    driver.get(url)

    # Thử nhấp chuột phải
    try:
        # Thực hiện nhấp chuột phải
        driver.execute_script("document.body.oncontextmenu = function() { return false; }")  # Tạm thời vô hiệu hóa nhấp chuột phải để kiểm tra

        # Nhấp chuột phải
        driver.execute_script("document.body.dispatchEvent(new MouseEvent('contextmenu', {bubbles: true}));")

        # Đợi một chút để kiểm tra có hiển thị menu không
        time.sleep(1)

        # Kiểm tra menu ngữ cảnh có hiển thị không
        context_menu_visible = driver.execute_script("return document.querySelector('body').style.contextMenu;")

        if context_menu_visible is None:  # Nếu không có menu ngữ cảnh hiển thị
            return 1  # Trả về 1, tức là nhấp chuột phải đã bị vô hiệu hóa
        else:
            return 0  # Trả về 0, tức là nhấp chuột phải vẫn hoạt động

    except Exception as e:
        print(f"Lỗi khi kiểm tra URL: {e}")
        return 0  # Trả về 0 nếu có lỗi
    finally:
        driver.quit()  # Đóng trình duyệt

# Ví dụ sử dụng
# url = "https://fithou.edu.vn"  # Thay đổi thành URL mà bạn muốn kiểm tra
# result = check_right_click_disabled(url)
# print(f"Kết quả kiểm tra vô hiệu hóa nhấp chuột phải cho '{url}': {result}")


#Cửa sổ bật lên của trình duyệt với các hộp văn bản



In [89]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException
import time

def check_browser_popup_with_textbox(url):
    # Thiết lập trình duyệt (sử dụng Chrome)
    driver = webdriver.Chrome()  # Đảm bảo rằng ChromeDriver đã được cài đặt
    driver.get(url)

    try:
        # Kiểm tra xem có cửa sổ bật lên không bằng cách kích hoạt một popup giả định
        driver.execute_script("window.prompt('Test prompt', '');")  # Tạo cửa sổ prompt giả để kiểm tra

        # Đợi một chút để cửa sổ bật lên xuất hiện
        time.sleep(1)

        # Kiểm tra xem cửa sổ bật lên có hiện không
        alert = driver.switch_to.alert

        # Nếu cửa sổ bật lên xuất hiện, kiểm tra xem nó có chứa hộp văn bản không
        if alert is not None:
            return 1  # Trả về 1, tức là có cửa sổ bật lên chứa hộp văn bản
    except UnexpectedAlertPresentException:
        return 1  # Trả về 1 nếu có cửa sổ bật lên chứa hộp văn bản
    except Exception as e:
        print(f"Lỗi khi kiểm tra URL: {e}")
        return 0  # Trả về 0 nếu có lỗi
    finally:
        driver.quit()  # Đóng trình duyệt

# Ví dụ sử dụng
# url = "https://fithou.edu.vn"  # Thay đổi thành URL mà bạn muốn kiểm tra
# result = check_browser_popup_with_textbox(url)
# print(f"Kết quả kiểm tra cửa sổ bật lên với hộp văn bản cho '{url}': {result}")


#Tên miền không có bản ghi DNS
pip install dnspython

In [90]:
pip install dnspython


In [91]:
import dns.resolver
import dns.exception

def check_dns_record(domain):
    try:
        # Thực hiện truy vấn DNS cho bản ghi A (hoặc có thể thay bằng các loại bản ghi khác)
        dns.resolver.resolve(domain, 'A')
        return 0  # Trả về 0 nếu có bản ghi DNS
    except dns.resolver.NoAnswer:
        return 1  # Trả về 1 nếu không có bản ghi DNS
    except dns.resolver.NXDOMAIN:
        return 1  # Trả về 1 nếu tên miền không tồn tại
    except Exception as e:
        print(f"Lỗi khi kiểm tra tên miền: {e}")
        return 1  # Trả về 1 nếu có lỗi

# Ví dụ sử dụng
# domain = "fithou.edu.vn"  # Thay đổi thành tên miền bạn muốn kiểm tra
# result = check_dns_record(domain)
# print(f"Kết quả kiểm tra bản ghi DNS cho '{domain}': {result}")


#Trang web không có chỉ mục của google
pip install requests beautifulsoup4

In [92]:
import requests
from bs4 import BeautifulSoup

def check_google_index(url):
    try:
        # Thay thế 'http://www.example.com' bằng tên miền bạn muốn kiểm tra
        search_url = f"https://www.google.com/search?q=site:{url}"

        # Gửi yêu cầu GET để lấy kết quả tìm kiếm
        response = requests.get(search_url)
        response.raise_for_status()  # Kiểm tra xem có lỗi trong yêu cầu không

        # Phân tích HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Kiểm tra xem có kết quả nào không
        if "did not match any documents" in soup.text:
            return 0  # Trả về 0 nếu không có chỉ mục
        else:
            return 1  # Trả về 1 nếu có chỉ mục

    except Exception as e:
        print(f"Lỗi khi kiểm tra chỉ mục: {e}")
        return 0  # Trả về 0 nếu có lỗi

#Ví dụ sử dụng
# url = "fithou.edu.vn"  # Thay đổi thành tên miền bạn muốn kiểm tra
# result = check_google_index(url)
# print(f"Kết quả kiểm tra chỉ mục cho '{url}': {result}")

25. #check_iframe

In [93]:
import requests
from bs4 import BeautifulSoup

def check_iframe(url):
    try:
        response = requests.get(url)
        # Kiểm tra nếu request thành công
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            # Tìm tất cả các thẻ <iframe>
            if soup.find('iframe'):
                return 1  # Có thẻ iframe
            else:
                return 0  # Không có thẻ iframe
        else:
            print("Không thể truy cập trang web.")
            return 0
    except requests.exceptions.RequestException as e:
        print("Lỗi khi truy cập URL:", e)
        return 0

# Ví dụ sử dụng
url = "https://docs.google.com/presentation/d/e/2PACX-1vQSm_Hl88CloSaw_LRl3AJrMQTkL3GCRfl2MSaMAz0vgGuUOuhFfhoom0kx54OUql8SvXJqk2_ZBGp_/pub?start=false&amp;loop=false&amp;delayms=3000"
print(check_iframe(url))

Không thể truy cập trang web.
0


26. #url of anchor

In [94]:
from urllib.parse import urlparse

def check_anchor(url):
    parsed_url = urlparse(url)
    # Kiểm tra phần fragment (nếu có) sau dấu #
    if parsed_url.fragment:
        return 1  # Có anchor
    else:
        return 0  # Không có anchor

# Ví dụ sử dụng
# url_with_anchor = "https://example.com/page#section1"
# url_without_anchor = "https://example.com/page"

# print(check_anchor(url_with_anchor))  # Kết quả: 1
# print(check_anchor(url_without_anchor))  # Kết quả: 0

27. #DNS record

In [95]:
pip install dnspython

In [96]:
import dns.resolver

def check_dns_record(domain):
    try:
        # Thực hiện truy vấn DNS cho bản ghi A (Address Record)
        dns.resolver.resolve(domain, 'A')
        return 1  # Có bản ghi DNS
    except dns.resolver.NoAnswer:
        return 0  # Không có bản ghi DNS
    except dns.resolver.NXDOMAIN:
        return 0  # Tên miền không tồn tại
    except Exception as e:
        # print("Lỗi khi kiểm tra DNS:", e)
        return 1  # Xử lý lỗi khác

# Ví dụ sử dụng
# domain = "https://docs.google.com/presentation/d/e/2PACX-1vQSm_Hl88CloSaw_LRl3AJrMQTkL3GCRfl2MSaMAz0vgGuUOuhFfhoom0kx54OUql8SvXJqk2_ZBGp_/pub?start=false&amp;loop=false&amp;delayms=3000"  # Thay thế bằng tên miền bạn muốn kiểm tra
# print(check_dns_record(domain))

28. #Abnormal URL

In [97]:
import re
from urllib.parse import urlparse

# def url_length(url):
#     """Tính chiều dài URL."""
#     return len(url)

def special_char_count(url):
    """Đếm số lượng ký tự đặc biệt trong URL."""
    return len(re.findall(r'[\?&=/#%]', url))

def is_strange_domain(url):
    """Kiểm tra xem tên miền có kỳ lạ hay không."""
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    return 1 if not re.match(r'^[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', domain) else 0

def param_count(url):
    """Đếm số lượng tham số trong URL."""
    return len(re.findall(r'[\?&]', url))

def contains_suspicious_keyword(url):
    """Kiểm tra sự hiện diện của từ khóa đáng ngờ trong URL."""
    suspicious_keywords = ['login', 'secure', 'account', 'update', 'verify']
    return 1 if any(keyword in url for keyword in suspicious_keywords) else 0

def subdomain_count(url):
    """Đếm số lượng subdomain trong tên miền."""
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    return domain.count('.') - 1  # Số dấu '.' trừ đi 1 cho tên miền chính

def contains_mailto(url):
    """Kiểm tra sự hiện diện của 'mailto:' trong URL."""
    return 1 if 'mailto:' in url else 0

def numeric_to_alpha_ratio(url):
    """Tính tỷ lệ ký tự số và chữ cái trong URL."""
    numeric_count = len(re.findall(r'\d', url))
    alpha_count = len(re.findall(r'[a-zA-Z]', url))
    return numeric_count / alpha_count if alpha_count > 0 else 0

# def extract_features(url):
#     """Tập hợp các đặc trưng từ các hàm riêng biệt."""
#     features = {
#         'url_length': url_length(url),
#         'special_char_count': special_char_count(url),
#         'is_strange_domain': is_strange_domain(url),
#         'param_count': param_count(url),
#         'contains_suspicious_keyword': contains_suspicious_keyword(url),
#         'subdomain_count': subdomain_count(url),
#         'contains_mailto': contains_mailto(url),
#         'numeric_to_alpha_ratio': numeric_to_alpha_ratio(url)
#     }
#     return features

# # Ví dụ sử dụng
# url = "https://example.com/login?user=test&secure=1"
# features = extract_features(url)
# print(features)


#Lấy cột url từ dataset chuyển vảo mảng để xử lý

In [98]:
import pandas as pd

# Đọc file CSV
df = pd.read_csv('phishing_new.csv')  # Thay 'file.csv' bằng đường dẫn tới file của bạn

# Lấy tất cả các URL từ cột 'url'
urls = df['url'].tolist()

# Kiểm tra kết quả
print(urls)

['https://qwn.pages.dev/', 'https://santos-99117.firebaseapp.com/', 'https://bafybeid2edyltg3b6pj5w25i37jzwmoe5a7wtn2sugkj2jcslzgvzls52m.ipfs.cf-ipfs.com/', 'https://onedrive.alicepierce9428.workers.dev/', 'http://1ebay-utente-notifica0nline.ueuo.com/', 'https://pub-ce910679e21d41d2b910a2656b19dcbf.r2.dev/index.html', 'https://docs.google.com/presentation/d/e/2PACX-1vQSm_Hl88CloSaw_LRl3AJrMQTkL3GCRfl2MSaMAz0vgGuUOuhFfhoom0kx54OUql8SvXJqk2_ZBGp_/pub?start=false&amp;loop=false&amp;delayms=3000', 'https://cloudflare-ipfs.com/ipfs/bafybeihsec7d2nswjsf34cnrqtcuwl53wcr47fwx5isdy3ap6k367dhiuu', 'https://desservermabagecloudsergeneratorjsjherjrsjgndf.pages.dev/', 'https://cloudflare-ipfs.com/ipfs/bafkreib4sdfsrkfq6mjhuz4awqpmtcz3c72p2irqvgfia7fttq53b6evl4', 'https://pub-57101e7d7e08497bb50a165ee30ff8d4.r2.dev/MSoutlook.htm', 'https://oneviseca.sviluppo.host', 'https://bafybeifjilrgbxbzztzp7z23hza7ccgtbqqsvytfpiktdjvqbx3m5unks4.ipfs.cf-ipfs.com/', 'https://klrn.wpenginepowered.com/klaaarnaa/kla

In [99]:
list1=[]

In [100]:
!pip install selenium # Cài đặt Selenium

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Trích xuất đặc trưng

In [101]:
i = 1
for url in urls:
    list2 = []  # Re-initialize list2 for each URL
    
    
    # 1. Check if URL contains '\.'
    list2.append(1 if check_url(url) == 1 else 0)
    
    # # 2. Độ dài phản hồi của một trang web
    # list2.append(response_length(url))
    
    # 3. kiểm tra xem có subdomain không
    list2.append(1 if count_subdomains(url) == 1 else 0)
     
    # 4. kiểm tra xem có tham số delta không
    list2.append(1 if check_delta_in_url(url) == 1 else 0)
    
    # 5. tính số lượng kí tự hex
    list2.append(hex_char_count(url))
    
    # 6. tính Số lượng kí tự hex/url
    list2.append(hex_ratio_hostname(url))
    
    # 7. dùng cổng http
    list2.append(1 if is_http(url) == 1 else 0)
    
    # 8. IP in URL
    list2.append(1 if is_ip_address(url) == 1 else 0)
    
    # 9. URL length
    list2.append(url_length(url))
    
    # 10. hostname length
    list2.append(calculate_hostname_length(url))
    
    # 11. Check for '@' symbol
    list2.append(1 if contains_at_symbol(url) == 1 else 0)
    
    # 12. số lượng chữ số / hostname
    list2.append(digit_ratio_in_hostname(url))
    
    # 13. độ đo entropy
    list2.append(check_hostname_entropy(url))
    
    # 14. tỷ lệ nguyên âm
    list2.append(calculate_vowel_ratio(url))
    
    # 15. tỷ lệ phụ âm
    list2.append(calculate_consonant_ratio(url))
    
    # # 16. Check xem url có trong google index không(quas nhiều url)
    # list2.append(1 if check_google_index(url) == 1 else 0)
    
    # 17. số lượng dấu '='
    list2.append(count_equal_signs(url))
    
    # 18. Chữ số/hostname
    list2.append(count_digits_in_hostname(url))
    
    # 19. Số lượng dấu ‘-’ / độ dài hostname
    list2.append(hyphen_ratio_in_hostname(url))
    
    # 20. Count number of dots in URL
    list2.append(count_dots(url))
    
    # 21. Check if URL uses priority port
    list2.append(1 if is_priority_port(url) == 1 else 0)
    
    # 22. Percentage of specific characters in URL
    list2.append(1 if percentage_of_characters(url) == 1 else 0)
    
    # 23. URL has hostname
    list2.append(1 if check_has_hostname(url) == 1 else 0)
    
    # 24. Số lượng dấu '/'
    list2.append(count_slashes(url))
    
    # # 25. URL has iframe
    # list2.append(1 if check_iframe(url) == 1 else 0)
    
    # 26. check_anchor
    list2.append(1 if check_anchor(url) == 1 else 0)
    
    # # 27. check_dns_record
    # list2.append(1 if check_dns_record(url) == 1 else 0)
    
    # 28. special_char_count
    list2.append(special_char_count(url))
    
    # 28. param_count
    list2.append(param_count(url))
    
    # 28. contains_suspicious_keyword
    list2.append(1 if contains_suspicious_keyword(url) == 1 else 0)
    
    # 28. subdomain_count
    list2.append(subdomain_count(url))
    
    # 28. contains_mailto
    list2.append(1 if contains_mailto(url) == 1 else 0)
    
    # 28. numeric_to_alpha_ratio
    list2.append(numeric_to_alpha_ratio(url))
    
    list2.append(1)
    
    # Add list2 to list1
    list1.append(list2)
    
    i += 1
    if i == 10000:
        break


In [102]:
feature_names = ['check_url','count_subdomains', 'check_delta_in_url', 'hex_char_count', 'hex_ratio_hostname','is_http','is_ip_address','url_length','calculate_hostname_length','contains_at_symbol','digit_ratio_in_hostname','check_hostname_entropy','calculate_vowel_ratio','calculate_consonant_ratio','count_equal_signs','count_digits_in_hostname','hyphen_ratio_in_hostname','count_dots','is_priority_port','percentage_of_characters','check_has_hostname','count_slashes','check_anchor','special_char_count','param_count','contains_suspicious_keyword','subdomain_count','contains_mailto','numeric_to_alpha_ratio','label']

In [103]:
# Create DataFrame
data = pd.DataFrame(list1, columns=feature_names)
data

,check_url,count_subdomains,check_delta_in_url,hex_char_count,hex_ratio_hostname,is_http,is_ip_address,url_length,calculate_hostname_length,contains_at_symbol,...,check_has_hostname,count_slashes,check_anchor,special_char_count,param_count,contains_suspicious_keyword,subdomain_count,contains_mailto,numeric_to_alpha_ratio,label
0,0,1,0,4,0.307692,0,0,22,13,0,...,1,1,0,3,0,0,1,0,0.000000,1
1,0,1,0,13,0.464286,0,0,37,28,0,...,1,1,0,3,0,0,1,0,0.200000,1
2,0,0,0,31,0.407895,0,0,85,76,0,...,1,1,0,3,0,0,2,0,0.222222,1
3,0,0,0,16,0.444444,0,0,45,36,0,...,1,1,0,3,0,0,2,0,0.117647,1
4,0,1,0,13,0.361111,1,0,44,36,0,...,1,1,0,3,0,0,1,0,0.058824,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,0,1,0,1,0.062500,0,0,25,16,0,...,1,1,0,3,0,0,1,0,0.000000,1
9995,0,0,0,13,0.476190,0,0,36,21,0,...,1,1,0,3,0,1,0,0,0.000000,1
9996,0,1,0,11,0.352941,0,0,36,17,0,...,1,2,0,4,0,0,1,0,0.037037,1
9997,0,1,0,37,0.837209,0,0,59,43,0,...,1,1,0,3,0,0,1,0,0.961538,1


In [104]:
data.to_csv('phishingfeature.csv', encoding='utf-8', index=False)
data

,check_url,count_subdomains,check_delta_in_url,hex_char_count,hex_ratio_hostname,is_http,is_ip_address,url_length,calculate_hostname_length,contains_at_symbol,...,check_has_hostname,count_slashes,check_anchor,special_char_count,param_count,contains_suspicious_keyword,subdomain_count,contains_mailto,numeric_to_alpha_ratio,label
0,0,1,0,4,0.307692,0,0,22,13,0,...,1,1,0,3,0,0,1,0,0.000000,1
1,0,1,0,13,0.464286,0,0,37,28,0,...,1,1,0,3,0,0,1,0,0.200000,1
2,0,0,0,31,0.407895,0,0,85,76,0,...,1,1,0,3,0,0,2,0,0.222222,1
3,0,0,0,16,0.444444,0,0,45,36,0,...,1,1,0,3,0,0,2,0,0.117647,1
4,0,1,0,13,0.361111,1,0,44,36,0,...,1,1,0,3,0,0,1,0,0.058824,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,0,1,0,1,0.062500,0,0,25,16,0,...,1,1,0,3,0,0,1,0,0.000000,1
9995,0,0,0,13,0.476190,0,0,36,21,0,...,1,1,0,3,0,1,0,0,0.000000,1
9996,0,1,0,11,0.352941,0,0,36,17,0,...,1,2,0,4,0,0,1,0,0.037037,1
9997,0,1,0,37,0.837209,0,0,59,43,0,...,1,1,0,3,0,0,1,0,0.961538,1
